# Resource Estimation of Random Circuit Used in State Vector Simulation

This notebook gives the resource estimation of random quantum circuits used for the full state vector simulation in
[Simulating 44-Qubit quantum circuits using AWS ParallelCluster](https://aws.amazon.com/blogs/hpc/simulating-44-qubit-quantum-circuits-using-aws-parallelcluster/).

In [ ]:
import qsharp

## Generating a random circuit

Let's now create a random Q# program. Note that the generated circuit will be different on each run, so resource estimates produced for it will vary slightly.

In [ ]:
%%qsharp
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Random;

operation RunRandomCircuit(nQubits : Int, nGates : Int) : Unit {
    use qs = Qubit[nQubits];

    for _ in 1 .. nGates {
        let rnd = DrawRandomDouble(0., 1.);
        if rnd < 0.5 {
            // CZ
            let controlInd = DrawRandomInt(0, nQubits - 1);
            // Make sure target qubit index is distinct from control qubit index
            mutable targetInd = DrawRandomInt(0, nQubits - 2);
            if targetInd >= controlInd {
                set targetInd += 1;
            }
            CZ(qs[controlInd], qs[targetInd]);
        } elif rnd < 0.625 {
            // H
            let ind = DrawRandomInt(0, nQubits - 1);
            H(qs[ind]);
        } else {
            // R
            let ind = DrawRandomInt(0, nQubits - 1);
            let gate = [Rx, Ry, Rz][DrawRandomInt(0, 2)];
            let angle = DrawRandomDouble(0., PI());
            gate(angle, qs[ind]);
        }
    }
}

## Running the Resource Estimator

Now we can estimate the resources required for the generated Q# program.

In [ ]:
n_qubits = 43
n_gates = 1000
qsharp.set_classical_seed(12)
result = qsharp.estimate(f"RunRandomCircuit({n_qubits}, {n_gates})")
result

In [ ]:
logical_qubits = result['physicalCounts']['breakdown']['algorithmicLogicalQubits']
logical_depth = result['physicalCounts']['breakdown']['algorithmicLogicalDepth']
# We take the runtime of the circuit from the blog post https://aws.amazon.com/blogs/hpc/simulating-44-qubit-quantum-circuits-using-aws-parallelcluster/, figure 4.
target_runtime = 5800
target_rqops = logical_qubits * logical_depth / target_runtime

print(f"Logical qubits = {logical_qubits}")
print(f"Logical depth = {logical_depth}")
print(f"Target runtime = {target_runtime} seconds")
print(f"Target rQOPS = {target_rqops}")

print(f"Execution time on hardware  = {result['physicalCounts']['runtime'] * 1e-9} seconds")
print(f"rQOPS for execution on hardware = {result['physicalCounts']['rqops']}")